In [1]:
from openai import OpenAI
import pandas as pd
import jellyfish
import numpy as np
import json
import os
import re
import time
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv()

True

In [3]:
_ = load_dotenv(find_dotenv()) # read local .env file

#openai.api_key = os.getenv("api.txt")
LONG_MODEL = "gpt-3.5-turbo-16k"
GPT_3 = "gpt-3.5-turbo"
GPT_4 = "gpt-4o"

In [4]:
os.chdir('/Users/chrissoria/Documents/Research/determinants-grad-adm')
current_directory = os.getcwd()
print(current_directory)

/Users/chrissoria/Documents/Research/determinants-grad-adm


In [5]:
def clean_school_names(df, column_name):
    df[column_name] = df[column_name].str.lower()
    df[column_name] = df[column_name].str.replace(r'[,-]', '', regex=True)
    df[column_name] = df[column_name].str.strip()
    df[column_name] = df[column_name].str.replace(r'\s+', ' ', regex=True)
    df[column_name] = df[column_name].str.replace(r'^the\s+', '', case=False, regex=True)
    return df

In [6]:
berkeley_schools = pd.read_excel('data/berkeley_schools.xlsx')
berkeley_schools = berkeley_schools[['UG Degree School']]
berkeley_schools = berkeley_schools.rename(columns = {'UG Degree School' : 'school'})
print(len(berkeley_schools))
berkeley_schools.head()

158007


,school
0,University of Kentucky
1,Technion-Israel Inst of Tech
2,"University of Minnesota, Twin Cities"
3,Univ of Nairobi
4,University of Southern California


In [7]:
berkeley_schools = clean_school_names(berkeley_schools, 'school')

berkeley_schools = pd.DataFrame(berkeley_schools['school'].dropna().unique(), columns=['school'])

print(len(berkeley_schools))
berkeley_schools.head()

5166


,school
0,university of kentucky
1,technionisrael inst of tech
2,university of minnesota twin cities
3,univ of nairobi
4,university of southern california


In [8]:
iped_schools = pd.read_csv('data/IPEDS_schools.csv')
iped_schools = iped_schools[['INSTNM']]
iped_schools = iped_schools.rename(columns = {'INSTNM' : 'school'})
print(len(iped_schools))
iped_schools.head()

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_17234/287998861.py:1: DtypeWarning: Columns (9,1537,1540,1542,1606,1608,1614,1615,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1703,1704,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1909,1910,1911,1912,1913,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1983,1984,2376,2377,2403,2404,2495,2496,2497,2498,2499,2500,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2527,2528,2529,2530,2958) have mixed types. Specify dtype option on import or set low_memory=False.
  iped_schools = pd.read_csv('data/IPEDS_schools.csv')


6543


,school
0,Alabama A & M University
1,University of Alabama at Birmingham
2,Amridge University
3,University of Alabama in Huntsville
4,Alabama State University


In [9]:
iped_schools = clean_school_names(iped_schools, 'school')

iped_schools = pd.DataFrame(iped_schools['school'].dropna().unique(), columns=['school'])

#for assesing a match rate I'll add a 1 to the df
iped_schools['match'] = 1

print(len(iped_schools))
iped_schools.head()

6405


,school,match
0,alabama a & m university,1
1,university of alabama at birmingham,1
2,amridge university,1
3,university of alabama in huntsville,1
4,alabama state university,1


There are 5222 schools in the berkeley list and 6543 in the IPEDS data. The highest possible max match is 5222. Therefore, in all future match rates we will set the denominator to 5222.

In [10]:
highest_possible_match = len(berkeley_schools)

Let's try and match this data based on the school column and see what match rate we can obtain.

In [11]:
merged_1 = berkeley_schools.merge(iped_schools, on= 'school', how = 'left')
merged_1.head()

,school,match
0,university of kentucky,1.0
1,technionisrael inst of tech,NaN
2,university of minnesota twin cities,NaN
3,univ of nairobi,NaN
4,university of southern california,1.0


In [12]:
print(f"The match rate with standardizing strings in both columns and nothing more is {merged_1['match'].sum() / highest_possible_match * 100:.2f}%")

The match rate with standardizing strings in both columns and nothing more is 23.83%


Next, let's try a fuzzy match based on a sufficiently high enough Jaro-Winkler score that that'll help us get around some of the small differences in way the school is spelled and also avoid false matches of schools with very similar names.

In [13]:
def find_best_match(school, choices, threshold=0.975):
    best_match = None
    highest_score = 0
    for choice in choices:
        score = jellyfish.jaro_winkler(school, choice)
        if score > highest_score:
            best_match = choice
            highest_score = score
    if highest_score >= threshold:
        return best_match, highest_score
    else:
        return None, highest_score

# Apply the matching function to each school name in berkeley_schools
matches = berkeley_schools['school'].apply(lambda x: find_best_match(x, iped_schools['school']))

# Create new columns for the best match and match score
berkeley_schools['Best Match'] = matches.apply(lambda x: x[0])
berkeley_schools['Match Score'] = matches.apply(lambda x: x[1])

berkeley_schools.head()

,school,Best Match,Match Score
0,university of kentucky,university of kentucky,1.000000
1,technionisrael inst of tech,None,0.828889
2,university of minnesota twin cities,university of minnesotatwin cities,0.982521
3,univ of nairobi,None,0.857179
4,university of southern california,university of southern california,1.000000


In [14]:
filtered_matches = berkeley_schools[berkeley_schools['Match Score'] >= 0.975]

merged_2 = filtered_matches.merge(iped_schools, left_on='Best Match', right_on='school', suffixes=('_left', '_right'), how='left')
merged_2.head()

,school_left,Best Match,Match Score,school_right,match
0,university of kentucky,university of kentucky,1.000000,university of kentucky,1
1,university of minnesota twin cities,university of minnesotatwin cities,0.982521,university of minnesotatwin cities,1
2,university of southern california,university of southern california,1.000000,university of southern california,1
3,brown university,brown university,1.000000,brown university,1
4,colorado school of mines,colorado school of mines,1.000000,colorado school of mines,1


In [15]:
print(f"The match rate with standardizing strings and a jaro-winkler threshold of .975 is {merged_2['match'].sum() / highest_possible_match * 100:.2f}%")

The match rate with standardizing strings and a jaro-winkler threshold of .975 is 26.93%


One problem here is that some of the schools in the berkeley list are international, or not a school at all. Let's start by asking GPT to identify which are real schools and which are domestic.

In [16]:
def identify_valid_schools(school_list,
                     user_model):
    
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    standardized_schools = []
    
    example_json = """{"country": "United States"}"""
    
    for school in school_list:
        prompt = f"""Tell me which country the college in triple backtics is located in: ```{school}```. \
        Put your response in JSON format with 'country' as the key and your output as the value. \
        If it's not a valid school make the value be 'invalid'. \
        Here's an example of what the JSON should look like: {example_json}"""
        try:
            response = client.chat.completions.create(
                model=user_model,
                response_format={"type": "json_object"},
                messages=[
                    {
                      "role": "system",
                      "content": f"""You provide direct and concise responses and provide only the answer to the question asked and provide only the requested JSON and nothing more."""
                    },
                    {'role': 'user', 
                     'content': prompt}
                ],
                temperature=0
            )


            standardized_school = response.choices[0].message.content
            standardized_schools.append(standardized_school)
            print(f"Processing row {school}")
        except Exception as e:
            print(f"An error occurred: {e}")
            standardized_schools.append(f"Error processing input: {school}")
            print('error tho')
            
    data = []
    
    for item in standardized_schools:
        parsed_json = json.loads(item)
        data.append(parsed_json)
        
    df = pd.DataFrame(data)
    standardized_schools = df['country']
    
    return standardized_schools

In [ ]:
berkeley_schools['country'] = identify_valid_schools(berkeley_schools['school'],
                                                          GPT_4)


In [18]:
berkeley_schools['domestic'] = berkeley_schools['country'].apply(lambda x: 1 if 'United States' in x else 0)
berkeley_schools.to_csv('data/berkeley_schools_features.csv')
berkeley_schools

,school,Best Match,Match Score,country,domestic
0,university of kentucky,university of kentucky,1.000000,United States,1
1,technionisrael inst of tech,None,0.828889,Israel,0
2,university of minnesota twin cities,university of minnesotatwin cities,0.982521,United States,1
3,univ of nairobi,None,0.857179,Kenya,0
4,university of southern california,university of southern california,1.000000,United States,1
...,...,...,...,...,...
5161,pec university of technology (deemed to be uni...,None,0.736349,India,0
5162,louisiana college,None,0.878834,United States,1
5163,niger,None,0.711111,invalid,0
5164,university of nebraska omaha,None,0.966359,United States,1


In [19]:
filtered_matches_domestic = berkeley_schools[berkeley_schools['Match Score'] >= 0.975]

merged_2 = filtered_matches.merge(iped_schools, left_on='Best Match', right_on='school', suffixes=('_left', '_right'), how='left')
merged_2.head()

,school_left,Best Match,Match Score,school_right,match
0,university of kentucky,university of kentucky,1.000000,university of kentucky,1
1,university of minnesota twin cities,university of minnesotatwin cities,0.982521,university of minnesotatwin cities,1
2,university of southern california,university of southern california,1.000000,university of southern california,1
3,brown university,brown university,1.000000,brown university,1
4,colorado school of mines,colorado school of mines,1.000000,colorado school of mines,1


Now, let's run these columns through an LLM to standardize the strings further and see what match rate we can get afterwards

In [20]:
def extract_standard(school_list,
                     user_model):
    
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    standardized_schools = []
    
    for school in school_list:
        prompt = f"""Please provide me with the correct and standard way of spelling the following college in triple backtics: ```{school}``` \
        put your response in JSON format with 'school' as the key and your output as the value."""
        try:
            response = client.chat.completions.create(
                model=user_model,
                response_format={"type": "json_object"},
                messages=[
                    {
                      "role": "system",
                      "content": f"""You provide direct and concise responses and provide only the answer to the question asked. \
                      You avoid using terms like, 'the standard way to spell this school is...' and provide only the standard way to spell the school."""
                    },
                    {'role': 'user', 
                     'content': prompt}
                ],
                temperature=0
            )


            standardized_school = response.choices[0].message.content
            standardized_schools.append(standardized_school)
            print(f"Processing row {school}")
        except Exception as e:
            print(f"An error occurred: {e}")
            standardized_schools.append(f"Error processing input: {school}")
            print('error tho')
            
    data = []
    
    for item in standardized_schools:
        parsed_json = json.loads(item)
        data.append(parsed_json)
        
    df = pd.DataFrame(data)
    standardized_schools = df['school']
    
    return standardized_schools

In [ ]:
berkeley_schools['gpt3_standardized'] = extract_standard(berkeley_schools['school'],
                                                          GPT_3)

berkeley_schools.head()

In [ ]:
iped_schools['gpt3_standardized'] = extract_standard(iped_schools['school'],
                                                          GPT_3)

In [24]:
matches_gpt = berkeley_schools['gpt3_standardized'].apply(lambda x: find_best_match(x, iped_schools['gpt3_standardized']))

berkeley_schools['Best Match GPT'] = matches_gpt.apply(lambda x: x[0])
berkeley_schools['Match Score GPT'] = matches_gpt.apply(lambda x: x[1])

berkeley_schools.head()

,school,Best Match,Match Score,country,domestic,gpt3_standardized,Best Match GPT,Match Score GPT
0,university of kentucky,university of kentucky,1.000000,United States,1,University of Kentucky,University of Kentucky,1.000000
1,technionisrael inst of tech,None,0.828889,Israel,0,Technion Israel Institute of Technology,None,0.806527
2,university of minnesota twin cities,university of minnesotatwin cities,0.982521,United States,1,University of Minnesota Twin Cities,University of Minnesota Twin Cities,1.000000
3,univ of nairobi,None,0.857179,Kenya,0,University of Nairobi,None,0.931746
4,university of southern california,university of southern california,1.000000,United States,1,University of Southern California,University of Southern California,1.000000


In [25]:
filtered_matches_gpt = berkeley_schools[berkeley_schools['Match Score GPT'] >= 0.975]

merged_3 = filtered_matches_gpt.merge(iped_schools, left_on='Best Match GPT', right_on='gpt3_standardized', suffixes=('_left', '_right'), how='left')
merged_3.head()

,school_left,Best Match,Match Score,country,domestic,gpt3_standardized_left,Best Match GPT,Match Score GPT,school_right,match,gpt3_standardized_right
0,university of kentucky,university of kentucky,1.000000,United States,1,University of Kentucky,University of Kentucky,1.0,university of kentucky,1,University of Kentucky
1,university of minnesota twin cities,university of minnesotatwin cities,0.982521,United States,1,University of Minnesota Twin Cities,University of Minnesota Twin Cities,1.0,university of minnesotatwin cities,1,University of Minnesota Twin Cities
2,university of southern california,university of southern california,1.000000,United States,1,University of Southern California,University of Southern California,1.0,university of southern california,1,University of Southern California
3,brown university,brown university,1.000000,United States,1,Brown University,Brown University,1.0,brown university,1,Brown University
4,colorado school of mines,colorado school of mines,1.000000,United States,1,Colorado School of Mines,Colorado School of Mines,1.0,colorado school of mines,1,Colorado School of Mines


In [26]:
print(f"The match rate with standardizing strings and a jaro-winkler threshold of .975 is {merged_3['match'].sum() / highest_possible_match * 100:.2f}%")

The match rate with standardizing strings and a jaro-winkler threshold of .975 is 31.32%


In [28]:
berkeley_schools.to_csv('data/berkeley_schools_features.csv', index = False)
iped_schools.to_csv('data/IPEDS_schools_features.csv', index = False)

The match rate is still not very high. But, the right-side data (IPEDS) only contains data on schools from the US, therefore using the total school denominator from the left-side data (Berkeley) is adequate. Recall, earlier we asked GPT to identify whether the school was located in the US. Let's take only those rows where the school is domestic and use that as the denominator and see what the true match rate is.

In [29]:
true_highest_possible_match = berkeley_schools['domestic'].sum()
print(f"GPT identified {true_highest_possible_match} domestic schools. This should be our denominator.")

GPT identified 2409 domestic schools. This should be our denominator.


In [30]:
print(f"The match rate with standardizing strings of domestic schools and a jaro-winkler threshold of .975 is {merged_3['match'].sum() / true_highest_possible_match * 100:.2f}%")

The match rate with standardizing strings of domestic schools and a jaro-winkler threshold of .975 is 67.16%


For my own exploration, here's a list of the domestic schools that I wasn't able to match

In [34]:
non_matches = berkeley_schools[berkeley_schools['domestic'] == 1]
non_matches = non_matches[non_matches['Match Score GPT'] < 0.975]

In [36]:
non_matches.to_csv('data/berkeley_ipeds_non_matches.csv', index=False)

Below, I'm producing a final matched dataset to send to Matthew for inspection

In [54]:
berkeley_baselines = pd.read_excel('data/berkeley_schools.xlsx')
berkeley_baselines = berkeley_baselines[['UG Degree School']]
print(len(berkeley_baselines))
berkeley_baselines.head()

158007


,UG Degree School
0,University of Kentucky
1,Technion-Israel Inst of Tech
2,"University of Minnesota, Twin Cities"
3,Univ of Nairobi
4,University of Southern California


In [55]:
berkeley_baselines = pd.DataFrame(berkeley_baselines['UG Degree School'].dropna().unique(), columns=['UG Degree School'])
berkeley_baselines['school_left'] = berkeley_baselines['UG Degree School']
berkeley_baselines = clean_school_names(berkeley_baselines, 'school_left')

print(len(berkeley_baselines))
berkeley_baselines.head()

5929


,UG Degree School,school_left
0,University of Kentucky,university of kentucky
1,Technion-Israel Inst of Tech,technionisrael inst of tech
2,"University of Minnesota, Twin Cities",university of minnesota twin cities
3,Univ of Nairobi,univ of nairobi
4,University of Southern California,university of southern california


In [49]:
berkeley_schools = pd.read_csv('data/berkeley_schools_features.csv')

In [39]:
merged_final = berkeley_schools.merge(
    iped_schools, 
    left_on='Best Match GPT', 
    right_on='gpt3_standardized', 
    suffixes=('_left', '_right'), 
    how='left'
)

print(len(merged_final))
merged_final.head()

5186


,school_left,Best Match,Match Score,country,domestic,gpt3_standardized_left,Best Match GPT,Match Score GPT,school_right,match,gpt3_standardized_right
0,university of kentucky,university of kentucky,1.000000,United States,1,University of Kentucky,University of Kentucky,1.000000,university of kentucky,1.0,University of Kentucky
1,technionisrael inst of tech,None,0.828889,Israel,0,Technion Israel Institute of Technology,None,0.806527,NaN,NaN,NaN
2,university of minnesota twin cities,university of minnesotatwin cities,0.982521,United States,1,University of Minnesota Twin Cities,University of Minnesota Twin Cities,1.000000,university of minnesotatwin cities,1.0,University of Minnesota Twin Cities
3,univ of nairobi,None,0.857179,Kenya,0,University of Nairobi,None,0.931746,NaN,NaN,NaN
4,university of southern california,university of southern california,1.000000,United States,1,University of Southern California,University of Southern California,1.000000,university of southern california,1.0,University of Southern California


In [56]:
merged_final = berkeley_baselines.merge(merged_final, on= 'school_left', how = 'left')

print(len(merged_final))
merged_final.head()

5957


,UG Degree School,school_left,Best Match,Match Score,country,domestic,gpt3_standardized_left,Best Match GPT,Match Score GPT,school_right,match,gpt3_standardized_right
0,University of Kentucky,university of kentucky,university of kentucky,1.000000,United States,1,University of Kentucky,University of Kentucky,1.000000,university of kentucky,1.0,University of Kentucky
1,Technion-Israel Inst of Tech,technionisrael inst of tech,None,0.828889,Israel,0,Technion Israel Institute of Technology,None,0.806527,NaN,NaN,NaN
2,"University of Minnesota, Twin Cities",university of minnesota twin cities,university of minnesotatwin cities,0.982521,United States,1,University of Minnesota Twin Cities,University of Minnesota Twin Cities,1.000000,university of minnesotatwin cities,1.0,University of Minnesota Twin Cities
3,Univ of Nairobi,univ of nairobi,None,0.857179,Kenya,0,University of Nairobi,None,0.931746,NaN,NaN,NaN
4,University of Southern California,university of southern california,university of southern california,1.000000,United States,1,University of Southern California,University of Southern California,1.000000,university of southern california,1.0,University of Southern California


In [59]:
iped_baselines = pd.read_csv('data/IPEDS_schools.csv')
iped_baselines = iped_baselines[['INSTNM','OPEID','OPEID6']]
iped_baselines['school_right'] = iped_baselines['INSTNM']

iped_baselines = clean_school_names(iped_baselines, 'school_right')

iped_baselines.head()

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_17234/917143819.py:1: DtypeWarning: Columns (9,1537,1540,1542,1606,1608,1614,1615,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1703,1704,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1909,1910,1911,1912,1913,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1983,1984,2376,2377,2403,2404,2495,2496,2497,2498,2499,2500,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2527,2528,2529,2530,2958) have mixed types. Specify dtype option on import or set low_memory=False.
  iped_baselines = pd.read_csv('data/IPEDS_schools.csv')


,INSTNM,OPEID,OPEID6,school_right
0,Alabama A & M University,100200.0,1002.0,alabama a & m university
1,University of Alabama at Birmingham,105200.0,1052.0,university of alabama at birmingham
2,Amridge University,2503400.0,25034.0,amridge university
3,University of Alabama in Huntsville,105500.0,1055.0,university of alabama in huntsville
4,Alabama State University,100500.0,1005.0,alabama state university


In [60]:
merged_final['school_right'] = merged_final.apply(lambda row: np.nan if row['Match Score GPT'] < 0.975 else row['school_right'], axis=1)

merged_final = merged_final.merge(iped_baselines, on= 'school_right', how = 'left')

merged_final.head()

,UG Degree School,school_left,Best Match,Match Score,country,domestic,gpt3_standardized_left,Best Match GPT,Match Score GPT,school_right,match,gpt3_standardized_right,INSTNM,OPEID,OPEID6
0,University of Kentucky,university of kentucky,university of kentucky,1.000000,United States,1,University of Kentucky,University of Kentucky,1.000000,university of kentucky,1.0,University of Kentucky,University of Kentucky,198900.0,1989.0
1,Technion-Israel Inst of Tech,technionisrael inst of tech,None,0.828889,Israel,0,Technion Israel Institute of Technology,None,0.806527,NaN,NaN,NaN,NaN,NaN,NaN
2,"University of Minnesota, Twin Cities",university of minnesota twin cities,university of minnesotatwin cities,0.982521,United States,1,University of Minnesota Twin Cities,University of Minnesota Twin Cities,1.000000,university of minnesotatwin cities,1.0,University of Minnesota Twin Cities,University of Minnesota-Twin Cities,396900.0,3969.0
3,Univ of Nairobi,univ of nairobi,None,0.857179,Kenya,0,University of Nairobi,None,0.931746,NaN,NaN,NaN,NaN,NaN,NaN
4,University of Southern California,university of southern california,university of southern california,1.000000,United States,1,University of Southern California,University of Southern California,1.000000,university of southern california,1.0,University of Southern California,University of Southern California,132800.0,1328.0


In [72]:
columns_to_keep = ['UG Degree School' ,'gpt3_standardized_left', 'INSTNM', 'OPEID', 'OPEID6', 'country', 'domestic']

schools_matched = merged_final[columns_to_keep].rename(columns={
    'UG Degree School': 'Berkeley School Name',
    'INSTNM': 'IPEDS School Name',
    'gpt3_standardized_left': 'Berkeley Correct School Name',
    'country': 'Estimated Country',
    'domestic': 'Estimated Domestic'
})

schools_matched['Estimated Country'] = schools_matched.apply(
    lambda row: 'United States' if pd.notna(row['OPEID']) else row['Estimated Country'], axis=1
)
schools_matched['Estimated Domestic'] = schools_matched.apply(
    lambda row: 1 if pd.notna(row['OPEID']) else row['Estimated Domestic'], axis=1
)

schools_matched.head()

,Berkeley School Name,Berkeley Correct School Name,IPEDS School Name,OPEID,OPEID6,Estimated Country,Estimated Domestic
0,University of Kentucky,University of Kentucky,University of Kentucky,198900.0,1989.0,United States,1
1,Technion-Israel Inst of Tech,Technion Israel Institute of Technology,NaN,NaN,NaN,Israel,0
2,"University of Minnesota, Twin Cities",University of Minnesota Twin Cities,University of Minnesota-Twin Cities,396900.0,3969.0,United States,1
3,Univ of Nairobi,University of Nairobi,NaN,NaN,NaN,Kenya,0
4,University of Southern California,University of Southern California,University of Southern California,132800.0,1328.0,United States,1


In [73]:
schools_matched.to_csv('data/berkeley_ipeds_matches.csv', index=False)